# Database connection

In [1]:
import pandas as pd
from decimal import *
from alex.database import Database # вспомогательный класс для подключения к базе и работе с sql
%matplotlib inline

In [2]:
# чтобы посмотреть содержимое файла database.py, из которого импортируется Database, раскомментируй следующую строку:
#%load alex/database/database.py

In [3]:
help(Database.query)

Help on function query in module alex.database.database:

query(self, sql)
    executes query against MSSQL, returns Pandas Dataframe



In [4]:
db = Database() # подключение к базе данных с учётными данными для Alex

In [5]:
def optimize_df(df):
    # оптимизация размера датафрейма - преобразование в категории того, чего надо 
    # и присвоение типа Decimal фин.показателям
    for col in df.columns:
        if col in ('exchange', 'pair', 'bid_ask'):
            df[col] = df[col].astype('category')
        if col in ('price', 'amount', 'volume'):
            df[col] = df[col].astype(Decimal)
    return df

## Most recent Order book

In [6]:
exchange = "binance"
pair = "ETH/USDT"

In [7]:
sql_orderbook = f"select * from v_order_book with (snapshot) where exchange='{exchange}' and pair='{pair}'"
df_orderbook = optimize_df(db.query(sql_orderbook))

In [8]:
df_orderbook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
id_ex_pair    200 non-null int64
exchange      200 non-null category
pair          200 non-null category
dt            200 non-null datetime64[ns]
bid_ask       200 non-null category
price         200 non-null object
amount        200 non-null object
volume        200 non-null object
dtypes: category(3), datetime64[ns](1), int64(1), object(3)
memory usage: 8.7+ KB


In [9]:
bids = sorted(df_orderbook[df_orderbook.bid_ask=='bid'].sort_values('price')['price'].values)
asks = sorted(df_orderbook[df_orderbook.bid_ask=='ask'].sort_values('price')['price'].values)
# rounding
bids = [round(x, 2) for x in bids]
asks = [round(x, 2) for x in asks]

In [10]:
size = min(len(bids), len(asks))
size

100

## Order book at particular date

In [11]:
date = '2018-09-20 0:00:00'
exchange = "binance"
pair = "ETH/USDT"
sql = f"select * from dbo.tvf_get_order_book_at_date('{date}') where exchange='{exchange}' and pair='{pair}'"
df_at_date = optimize_df(db.query(sql))

In [12]:
df_at_date.sort_values('dt', ascending=False).head()

,id_ex_pair,exchange,pair,dt,bid_ask,price,amount,volume
15,19,binance,ETH/USDT,2018-09-19 23:59:58,bid,211.08,5.54241,1169.89
150,19,binance,ETH/USDT,2018-09-19 23:59:58,ask,214.47,0.8,171.576
124,19,binance,ETH/USDT,2018-09-19 23:59:58,bid,208.11,1.501,312.373
55,19,binance,ETH/USDT,2018-09-19 23:59:58,ask,211.23,13.3108,2811.63
17,19,binance,ETH/USDT,2018-09-19 23:59:58,bid,211.1,42.713,9016.72


## Order book snapshots for last N minutes

In [13]:
minutes = 120 # кол-во минут назад от текущего момента
exchange = "binance"
pair = "ETH/USDT"
sql = f"select * from tvf_get_order_book_snapshots({minutes}) where exchange='{exchange}' and pair='{pair}'"
df_orderbook_snapshots = optimize_df(db.query(sql))

In [14]:
df_orderbook_snapshots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74600 entries, 0 to 74599
Data columns (total 7 columns):
dt          74600 non-null datetime64[ns]
exchange    74600 non-null category
pair        74600 non-null category
bid_ask     74600 non-null category
price       74600 non-null object
amount      74600 non-null object
volume      74600 non-null object
dtypes: category(3), datetime64[ns](1), object(3)
memory usage: 2.5+ MB


## History

In [29]:
date = '2018-09-20 0:00:00' # дата с которой загружать историю
sql_history = f"select * from v_history with (snapshot) where exchange='{exchange}' and pair='{pair}' " \
              f"and dt>='{date}'"

In [30]:
df_history = optimize_df(db.query(sql_history))

In [31]:
df_history.set_index('dt', inplace=True)

In [32]:
df_history.head()

,id_ex_pair,exchange,pair,price,amount,type,side,id,location,insert_date
dt,,,,,,,,,,
2018-09-20 02:35:02,19,binance,ETH/USDT,209.09,6.99656,None,buy,36389444.0,memory,2018-09-20 02:35:03.7800000
2018-09-20 02:35:02,19,binance,ETH/USDT,209.09,0.90281,None,sell,36389443.0,memory,2018-09-20 02:35:03.7800000
2018-09-20 02:35:07,19,binance,ETH/USDT,209.09,2.34124,None,sell,36389445.0,memory,2018-09-20 02:35:09.4033333
2018-09-20 02:35:07,19,binance,ETH/USDT,209.08,4.87118,None,sell,36389446.0,memory,2018-09-20 02:35:09.4033333
2018-09-20 02:35:12,19,binance,ETH/USDT,209.17,0.52498,None,buy,36389447.0,memory,2018-09-20 02:35:14.7100000
